# 데이터 전처리 과정

In [1]:
import pandas as pd
import os

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

# MF model 구성

In [7]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [8]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=1024, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# 데이터 분석

In [9]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

print('ratings에 있는 유니크한 영화 개수{}'.format(len(movie_unique)))
print('ratings에 있는 유니크한 사용자 수: {}'.format(len(user_unique)))

ratings에 있는 유니크한 영화 개수3628
ratings에 있는 유니크한 사용자 수: 6039


In [10]:
# print('가장 인기 있는 영화 30개(인기순):')

# 내가 선호하는 영화 5가지 추가

In [11]:
# ratings & movies 합쳐서 내 정보 추가할 수 있는 새로운 데이터 생성
data = ratings.join(movies.set_index("movie_id"), on="movie_id")
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [12]:
data.drop(["timestamp", "movie_id", "genre"], axis=1, inplace=True)
data.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"


In [13]:
my_favorites = [
    'Notting Hill (1999)',
    'Meet the Parents (2000)',
    'Sleepless in Seattle (1993)',
    "You've Got Mail (1998)",
    'Sabrina (1995)'
]

data= data.append(pd.DataFrame({
    'user_id': ['heej']*5,
    'title': my_favorites,
    'counts':[5] * 5}))

In [14]:
data.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"


In [15]:
data.tail()

,user_id,counts,title
0,heej,5,Notting Hill (1999)
1,heej,5,Meet the Parents (2000)
2,heej,5,Sleepless in Seattle (1993)
3,heej,5,You've Got Mail (1998)
4,heej,5,Sabrina (1995)


In [16]:
data= data.reset_index()
data= data.drop(['index'],axis=1)
data.tail()

,user_id,counts,title
836478,heej,5,Notting Hill (1999)
836479,heej,5,Meet the Parents (2000)
836480,heej,5,Sleepless in Seattle (1993)
836481,heej,5,You've Got Mail (1998)
836482,heej,5,Sabrina (1995)


In [17]:
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

In [18]:
user_unique

array([1, 2, 3, ..., 6039, 6040, 'heej'], dtype=object)

In [19]:
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')


data

user_id column indexing OK!!


,user_id,counts,title
0,0,5,One Flew Over the Cuckoo's Nest (1975)
1,0,3,James and the Giant Peach (1996)
2,0,3,My Fair Lady (1964)
3,0,4,Erin Brockovich (2000)
4,0,5,"Bug's Life, A (1998)"
...,...,...,...
836478,6039,5,Notting Hill (1999)
836479,6039,5,Meet the Parents (2000)
836480,6039,5,Sleepless in Seattle (1993)
836481,6039,5,You've Got Mail (1998)


In [21]:
# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['title'] = temp_artist_data
else:
    print('artist column indexing Fail!!')
    
data

artist column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
836478,6039,5,1151
836479,6039,5,461
836480,6039,5,666
836481,6039,5,626


# CSR matrix 생성
user X movie title 에 대한 CSR 행렬 생성

In [22]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

In [23]:
len(user_unique)

6040

In [24]:
len(movie_unique)

3628

In [25]:
user_to_idx['heej']

6039

In [26]:
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

In [27]:
data.tail()

,user_id,counts,title
836478,6039,5,1151
836479,6039,5,461
836480,6039,5,666
836481,6039,5,626
836482,6039,5,516


In [28]:
csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [29]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

# 모델 훈련
    factors=1024
    regularization=0.01
    use_gpu=False
    iterations=15
    dtype=np.float32

In [30]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 모델의 예측 선호도 파악

#### heej 벡터

In [31]:
heej_idx= user_to_idx["heej"]
heej_vector = als_model.user_factors[heej_idx]

#### 선호 영화 벡터

In [32]:
nottinghill_vector = als_model.item_factors[movie_to_idx['Notting Hill (1999)']]
meettheparents_vector = als_model.item_factors[movie_to_idx['Meet the Parents (2000)']]
sleeplessinceattle_vector = als_model.item_factors[movie_to_idx['Sleepless in Seattle (1993)']]
youvegotmail_vector = als_model.item_factors[movie_to_idx["You've Got Mail (1998)"]]
sabrina_vector = als_model.item_factors[movie_to_idx['Sabrina (1995)']]

In [33]:
np.dot(heej_vector, nottinghill_vector)

0.9324743

In [34]:
np.dot(heej_vector, meettheparents_vector)

0.9546996

In [35]:
np.dot(heej_vector, sleeplessinceattle_vector)

0.95260274

In [36]:
np.dot(heej_vector, youvegotmail_vector)

0.9440406

In [37]:
np.dot(heej_vector, sabrina_vector)

0.84458095

#### 비선호 영화 벡터

In [38]:
contender_vector = als_model.item_factors[movie_to_idx['Contender, The (2000)']]

In [39]:
np.dot(heej_vector, contender_vector)

0.032612327

# 제시하는 영화와 비슷한 영화를 추천

In [40]:
movie_list= list(movie_to_idx.keys())
idx_list= list(movie_to_idx.values())

In [41]:
movie_id = movie_to_idx['Notting Hill (1999)']
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1151, 1.0),
 (3622, 0.5086692),
 (3268, 0.5082949),
 (3508, 0.5073469),
 (3381, 0.5071988),
 (3626, 0.50713),
 (3498, 0.507118),
 (3405, 0.5070613),
 (3410, 0.50664645),
 (3494, 0.50590265),
 (3201, 0.5054197),
 (3543, 0.5054171),
 (3324, 0.5054067),
 (3472, 0.505176),
 (3535, 0.5051639)]

In [42]:
for i, vector_value in similar_movie:
    idx_list.index(i)
    
    print(movie_list[idx_list.index(i)], vector_value)

Notting Hill (1999) 1.0
Master Ninja I (1984) 0.5086692
Grosse Fatigue (1994) 0.5082949
I, Worst of All (Yo, la peor de todas) (1990) 0.5073469
Mascara (1999) 0.5071988
Five Wives, Three Secretaries and Me (1998) 0.50713
Savage Nights (Nuits fauves, Les) (1992) 0.507118
Carriers Are Waiting, The (Les Convoyeurs Attendent) (1999) 0.5070613
Foolish (1999) 0.50664645
An Unforgettable Summer (1994) 0.50590265
Allnighter, The (1987) 0.5054197
Small Wonders (1996) 0.5054171
Race the Sun (1996) 0.5054067
Show, The (1995) 0.505176
Modern Affair, A (1995) 0.5051639


# 사용자가 좋아할 만한 영화 추천

In [43]:
user = user_to_idx['heej']
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)

In [44]:
# movie_list= list(movie_to_idx.keys())
# idx_list= list(movie_to_idx.values())

In [45]:
for i, vector_value in movie_recommended:
    idx_list.index(i)
    
    print(movie_list[idx_list.index(i)], vector_value)

While You Were Sleeping (1995) 0.16697016
It Could Happen to You (1994) 0.1289986
Pretty Woman (1990) 0.10957982
French Kiss (1995) 0.09440611
One Fine Day (1996) 0.09317218
American President, The (1995) 0.08725761
Mirror Has Two Faces, The (1996) 0.08467703
Far and Away (1992) 0.083415665
Runaway Bride (1999) 0.079137005
Tin Cup (1996) 0.07773541


## 결과
#### 1) 데이터 사이즈와 CSR matrix
    - ratings에 있는 유니크한 영화 개수3628
    - ratings에 있는 유니크한 사용자 수: 6040 (개인 선호도 추가한 내용 포함)
    - csr_data: 6040x3628 sparse matrix 

#### 2) 추가한 사용자 데이터의 사용자와 영화 벡터 내적 수치: 0.8 ~ 0.9
    model 의 factor size 가 100이었을 때에는 각각의 영화와 사용자 벡터 간의 내적 수치가 약 0.4 정도로, 의미 있는 결과를 얻을 수 없었으나, factor 를 1024로 늘렸을 때에는 각 벡터의 내적 수치가 0.8~0.9 로 상향되어 의미 있는 결과값을 얻었다.

#### 3) MF 모델의 similar, recommend 로 예측한 결과
    [1] 제시하는 영화와 비슷한 영화를 추천한 결과를 보면, 각각의 영화와 제시한 영화의 벡터간 내적 수치는 약 0.48, [2] 사용자가 좋아할 만한 영화는 약 0.1 정도로 2) 에서 계산한 사용자와 그 사용자가 선호하는 영화 벡터의 내적값에 비하면 매우 낮은 값이 나왔다.

#### 회고
    모델이 마지막에 추가한 데이터에 대해서는 높은 값의 벡터 내적값으로 의미있는 결과를 잘 낼 수 있도록 training 되었다고 보여 졌으나 이에 비해 직접 제시하지 않은 문제는 잘 풀지 못하는 것으로 보아 overfitting 이 의심 된다. 제공된 데이터를 이용하여 장르별 추천 등 새롭게 새도해 볼 수 있는 부분이 보여 흥미로웠다. 위의 3) 에서 더 좋은 결과값을 받을 수 있도록 모델을 변경하여 학습이 잘 되면 이 부분은 더 해볼 수 있겠다.